# scratch work

In [43]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

from jorbit.observation import Observations
from jorbit.utils.states import SystemState
from jorbit.ephemeris import Ephemeris
from jorbit.accelerations import create_newtonian_ephemeris_acceleration_func
from jorbit.integrators import ias15_evolve, initialize_ias15_integrator_state
from jorbit.astrometry.sky_projection import on_sky, sky_sep

t0 = Time("2024-12-01 00:00")
t1 = Time("2025-12-01 00:00")

In [44]:
obj = Horizons(id="274301", location="@0", epochs=[t0.tdb.jd, t1.tdb.jd])
vecs = obj.vectors(refplane="earth")

x0 = jnp.array([vecs["x"][0], vecs["y"][0], vecs["z"][0]])
v0 = jnp.array([vecs["vx"][0], vecs["vy"][0], vecs["vz"][0]])

x1 = jnp.array([vecs["x"][-1], vecs["y"][-1], vecs["z"][-1]])
v1 = jnp.array([vecs["vx"][-1], vecs["vy"][-1], vecs["vz"][-1]])


obj = Horizons(id="274301", location="695", epochs=[t0.jd, t1.jd])
eph = obj.ephemerides(extra_precision=True, quantities="1")
coord0 = SkyCoord(eph["RA"][0], eph["DEC"][0], unit=(u.deg, u.deg), frame="icrs")
coord1 = SkyCoord(eph["RA"][-1], eph["DEC"][-1], unit=(u.deg, u.deg), frame="icrs")

In [45]:
obs = Observations(
    observed_coordinates=[coord0, coord0],
    times=jnp.array([t0.tdb.jd, t1.tdb.jd]),
    observatories="695@399",
    astrometric_uncertainties=1 * u.arcsec,
    verbose=True,
)

eph = Ephemeris(
    earliest_time=Time("2015-01-01 00:00"),
    latest_time=Time("2035-01-01 00:00"),
    ssos="default planets",
)

acc_func = create_newtonian_ephemeris_acceleration_func(eph.processor)

state0 = SystemState(
    positions=jnp.array([x0]),
    velocities=jnp.array([v0]),
    log_gms=jnp.array([0.0]),
    time=t0.tdb.jd,
    acceleration_func_kwargs=None,
)

ra0, dec0 = on_sky(state0, acc_func, obs.observer_positions[0])
calc_coord0 = SkyCoord(ra0, dec0, unit=(u.rad, u.rad), frame="icrs")
calc_coord0.separation(coord0)

<Angle [3.19443392e-10] deg>

In [50]:
new_positions, state1, new_integrator_state = ias15_evolve(
    initial_system_state=state0,
    acceleration_func=acc_func,
    times=jnp.array([t1.tdb.jd]),
    initial_integrator_state=initialize_ias15_integrator_state(acc_func(state0)),
    n_steps=100,
)

ra1, dec1 = on_sky(state1, acc_func, obs.observer_positions[1])
calc_coord1 = SkyCoord(ra1, dec1, unit=(u.rad, u.rad), frame="icrs")
calc_coord1.separation(coord1)

<Angle [5.47304387e-07] deg>

In [47]:
state1.positions, state0.positions

(Array([[-2.17856334, -1.13866228, -0.35793294]], dtype=float64),
 Array([[-1.78355289,  1.97281979,  0.57967129]], dtype=float64))

In [48]:
x1, x0

(Array([-2.17856337, -1.13866227, -0.35793294], dtype=float64),
 Array([-1.78355289,  1.97281979,  0.57967129], dtype=float64))

In [49]:
new_integrator_state.dt

Array(28.87082204, dtype=float64)